# CNN to dog breeds

In this notebook, I aim to apply Tensorflow and Pytorch AI frameworks to classify different Dog breed given an image.

## Data

In [14]:
# Import libraries
import pandas as pd
import glob

In [15]:
# Load the dog images (.jpg extension)
dog_images = glob.glob("dogimages/**/*.jpg", recursive=True)
num_dog_images = len(dog_images)
print(f"There are {num_dog_images} dog images.")

There are 8351 dog images.


In [16]:
# Train
train_dog_images = glob.glob("dogimages/train/**/*.jpg", recursive=True)
num_train_dog_images = len(train_dog_images)
print(f"There are {num_train_dog_images} dog images for train.")

There are 6680 dog images for train.


In [17]:
# Validation
val_dog_images = glob.glob("dogimages/valid/**/*.jpg", recursive=True)
num_val_dog_images = len(val_dog_images)
print(f"There are {num_val_dog_images} dog images for validation.")

There are 835 dog images for validation.


In [18]:
# Test
test_dog_images = glob.glob("dogimages/test/**/*.jpg", recursive=True)
num_test_dog_images = len(test_dog_images)
print(f"There are {num_test_dog_images} dog images for test.")

There are 836 dog images for test.


In [19]:
# We check that there is no any wrong or missing image
assert num_dog_images == num_train_dog_images + num_val_dog_images + num_test_dog_images

In [20]:
# Different breeds
dog_breeds = glob.glob("dogimages/train/*")
dog_breeds = [dog_breed_tmp.split(".")[1] for dog_breed_tmp in dog_breeds]

**TODO**: Show some image

## CNN

In [21]:
# Import libraries
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np

### Tensorflow

In [22]:
# Import libraries
import tensorflow as tf

**Step 1**: Load data

In [ ]:
# Image size redimension
IMG_SIZE = (128, 128)
# Images per Batch (bigger = more RAM)
BATCH_SIZE = 32
# Carga las imágenes desde la carpeta "data/train"
train_ds = tf.keras.utils.image_dataset_from_directory(
    "dogimages/train",      # Path
    image_size=IMG_SIZE,    # Image redimension -> 128x128
    batch_size=BATCH_SIZE,  # Groups (batchs) of 32 images
    shuffle=True            # To improve the training process
)
# Same for validation
val_ds = tf.keras.utils.image_dataset_from_directory(
    "dogimages/valid",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)
# Same for test
test_ds = tf.keras.utils.image_dataset_from_directory(
    "dogimages/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)
# We automatically obtain the classes
class_names = train_ds.class_names
print("Clases encontradas:", class_names)

Found 6680 files belonging to 133 classes.
Found 835 files belonging to 133 classes.
Found 836 files belonging to 133 classes.
Clases encontradas: ['001.Affenpinscher', '002.Afghan_hound', '003.Airedale_terrier', '004.Akita', '005.Alaskan_malamute', '006.American_eskimo_dog', '007.American_foxhound', '008.American_staffordshire_terrier', '009.American_water_spaniel', '010.Anatolian_shepherd_dog', '011.Australian_cattle_dog', '012.Australian_shepherd', '013.Australian_terrier', '014.Basenji', '015.Basset_hound', '016.Beagle', '017.Bearded_collie', '018.Beauceron', '019.Bedlington_terrier', '020.Belgian_malinois', '021.Belgian_sheepdog', '022.Belgian_tervuren', '023.Bernese_mountain_dog', '024.Bichon_frise', '025.Black_and_tan_coonhound', '026.Black_russian_terrier', '027.Bloodhound', '028.Bluetick_coonhound', '029.Border_collie', '030.Border_terrier', '031.Borzoi', '032.Boston_terrier', '033.Bouvier_des_flandres', '034.Boxer', '035.Boykin_spaniel', '036.Briard', '037.Brittany', '038.Bru

**Step 2**: Normalizar imágenes y preparar el pipeline de datos

In [ ]:
# TensorFlow can automatically optimize the number of threads when loading data
AUTOTUNE = tf.data.AUTOTUNE

# Pixels from inter [0,255] to float [0.0,1.0]
def preprocess(ds):
    return ds.map(
        lambda x, y: (tf.cast(x, tf.float32) / 255.0, y),  # Image normalizarion
        num_parallel_calls=AUTOTUNE
    )

# Train
train_ds = (
    preprocess(train_ds)             # Image normalization
    .cache()                         # Save data after first usage
    .shuffle(1000)                   # Shuffle to improve the training process
    .prefetch(buffer_size=AUTOTUNE)  # Preparing next batch whereas this is training (remember: parallel mode is active)
)

# Validation
val_ds = (
    preprocess(val_ds)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

# Test
test_ds = (
    preprocess(test_ds)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [ ]:
# Secuencial CNN: layer by layer
model = tf.keras.models.Sequential([
    # First, we define the input as the image size
    tf.keras.layers.Input(shape=(*IMG_SIZE, 3)),  # Imagen de entrada: (128, 128, 3)
    # Second, we define 3 blocks Conv2D +  MaxPooling2D to apply filters and reduce the image, respectively.
    ## 1sd Conv block: 32 filters + ReLU
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),  # (128x128) -> (64x64)
    ## 2nd Conv block: 64 filters + ReLU
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),  # (64x64) -> (32x32)
    ## 3rd Conv block: 128 filters + ReLU
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),  # (32x32) -> (16x16)
    # Image to 1D
    tf.keras.layers.Flatten(),
    # Combine lerant features
    tf.keras.layers.Dense(128, activation='relu'),
    # Regularization: power off 50% of neurons for avoid overfitting
    tf.keras.layers.Dropout(0.5),
    # Output layer: number of neurons = number of classes -> softmax (probabilities)
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

# Compiling the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Show the model
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 133)            │        17,157 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,321,797 (12.67 MB)

 Trainable params: 3,321,797 (12.67 MB)

 Non-trainable params: 0 (0.00 B)

**Step 4**: Train the model

In [27]:
# Train the model: train a validation datasets
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
208/209 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.0076 - loss: 4.8944

c:\Users\alvar\miniconda3\envs\dog-env\lib\site-packages\keras\src\trainers\epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


209/209 ━━━━━━━━━━━━━━━━━━━━ 33s 159ms/step - accuracy: 0.0076 - loss: 4.8943 - val_accuracy: 0.0251 - val_loss: 4.8261
Epoch 2/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 32s 154ms/step - accuracy: 0.0174 - loss: 4.7969 - val_accuracy: 0.0299 - val_loss: 4.6646
Epoch 3/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 32s 154ms/step - accuracy: 0.0229 - loss: 4.6784 - val_accuracy: 0.0311 - val_loss: 4.6230
Epoch 4/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 32s 151ms/step - accuracy: 0.0343 - loss: 4.5854 - val_accuracy: 0.0371 - val_loss: 4.4647
Epoch 5/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 32s 154ms/step - accuracy: 0.0405 - loss: 4.4503 - val_accuracy: 0.0539 - val_loss: 4.3370
Epoch 6/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 32s 153ms/step - accuracy: 0.0538 - loss: 4.2795 - val_accuracy: 0.0683 - val_loss: 4.2175
Epoch 7/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 32s 152ms/step - accuracy: 0.0657 - loss: 4.1373 - val_accuracy: 0.0707 - val_loss: 4.1708
Epoch 8/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 33s 156ms/step - accuracy: 0.0858 - loss: 4.0001 - val

**Step 5**: Test the model

In [28]:
# Test the test data (without being used until now)
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc:.2%}")

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.0884 - loss: 4.0054
Test Accuracy: 9.09%


### Pytorch